In [ ]:
import requests
from PIL import Image
import requests
import argparse
import csv
import base64
import sys
from io import BytesIO
from PIL import Image
import six
import shutil
import uuid
from os import path as pth
import os
import multiprocessing
import time

In [ ]:
def scale(content, min_dim):


    image = Image.open(content)

    # no scaling, keep images full size
    if min_dim == -1:
        return image

    # aspect-ratio preserving scale so that the smallest dimension is `min_dim`
    width, height = image.size
    scale_dimension = width if width < height else height
    scale_ratio = float(min_dim) / scale_dimension

    if scale_ratio == 1:
        return image

    return image.resize(
        (int(width * scale_ratio), int(height * scale_ratio)),
        Image.ANTIALIAS,
    )


def read_image(response, min_dim=0):
    """ Download response in chunks and convert to a scaled Image object """

    content = BytesIO(response)
    content.seek(0)
    
    if min_dim > 0 :
        return scale(content, min_dim)
    else:
        return content


In [ ]:
def get_data(text, save_dir, nb_images=10):
    try:
        r = requests.get("https://api.qwant.com/api/search/images",
            params={
                'count': nb_images,
                'q': text,
                't': 'images',
                'safesearch': 1,
                'locale': 'fr_FR',
                'uiv': 4
            },
            headers={
                'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/56.0.2924.87 Safari/537.36'
            },
            stream=True, timeout=2.0
        )
    except Exception as e:
        print(e)
        return
    try:
        response = r.json().get('data').get('result').get('items')
    except Exception as e:
        print(e)
        return
    urls = [r.get('media') for r in response]
    for url in urls:
        id = str(uuid.uuid5(uuid.NAMESPACE_DNS, url)) + '.jpg'
        if 'base64,' in url:
            image_data = url.split(',')[1]
            buffered = BytesIO()
            im = Image.open(buffered, format="JPEG").convert("RGB")
            im.save(pth.join(save_dir,id))
        else:
            try:
                response = requests.get(url,stream=True, timeout=2.0)
            except Exception as e:
                print(e)
                continue
            if response.status_code < 400:
                try :
                    im = Image.open(read_image(response.content)).convert("RGB")
                    im.save(pth.join(save_dir,id))
                except Exception as e:
                    print(e)
    return

In [ ]:
def consumer(query):
    while queue.empty():
        time.sleep(5.0)  # give the queue a chance to populate
    
    while not queue.empty():
        q = queue.get(block=True, timeout=None)
        save_dir = "/data/datasets/QwantCrawl/"+q
        if (not pth.isdir(save_dir)):
            os.mkdir(save_dir)
        get_data(q, save_dir)
        
        if queue.empty():
            time.sleep(0.1)
    print("Exited Consumer")

In [ ]:
def producer(queries, queue):
    f = False
    for q in queries:
        print(q)
        queue.put(q, block=True, timeout=None)

In [ ]:
f = open("/data/datasets/log_images.csv")
reader = csv.reader(f)
queries = set()
for line in reader:
    if len(line) > 3:
        print(line)
    queries.add(line[1].replace('/', ' '))
print("Queries ", len(queries))
queue = multiprocessing.Queue(1000)
processes = [
    multiprocessing.Process(target=producer, args=(queries,queue))
]

for i in range(10):
    processes.append(multiprocessing.Process(target=consumer, args=[queue]))
print('There is :', len(processes), 'processes' )

for p in processes:
    p.start()

for p in processes:
    p.join()